Cuardeno para procesar la data cruda del saber tyt de DataICFES

### Notas:

En la actualizacion del 31/07/2025 de los resultados, el icfes no incluyó la columna punt_global, por lo cual se calculó en este notebook con base en:

https://www.icfes.gov.co/wp-content/uploads/2025/03/02_entender_mis_resultados.pdf

### Constantes y Librerias 

In [1]:
import os
import glob
import pandas as pd
from unidecode import unidecode
import numpy as np
import csv
#Archivo con constantes
import constants

### Funciones

In [2]:
def calcular_punt_global(df):
    cols = [
        'mod_ingles_punt',
        'mod_competen_ciudada_punt',
        'mod_comuni_escrita_punt',
        'mod_razona_cuantitat_punt',
        'mod_lectura_critica_punt'
    ]
    #hacemos fill na porque hay unos puntajes sin calificacion
    df['punt_global'] = df[cols].fillna(0).mean(axis=1)
    return df

In [3]:
def resumen_nulos(df):
    cols = [
        'mod_ingles_punt',
        'mod_competen_ciudada_punt',
        'mod_comuni_escrita_punt',
        'mod_razona_cuantitat_punt',
        'mod_lectura_critica_punt'
    ]
    
    resumen = (
        df[cols]
        .isna()
        .agg(['sum', 'mean'])
        .T
        .rename(columns={'sum': 'n_nulos', 'mean': 'porc_nulos'})
    )
    
    resumen['porc_nulos'] = resumen['porc_nulos'] * 100  # convert to percentage
    return resumen


### Lectura informacion cruda del saber 11

In [4]:
#Directorios con la datos del saber tyt
data_dir_base_sbtyt = "../../data/SABERTYT_raw/actualizacion_31072025"
#lista con la ruta de los archivos 
csv_files_sabertyt = glob.glob(os.path.join(data_dir_base_sbtyt, "*.txt"))

In [5]:
#Variables a seleccionar del saber tyt 
# Convertimos a set para búsquedas más rápidas
cols_set = set(constants.cols_saber_tyt_lower)

In [6]:
# Lista donde se irán acumulando los DataFrames leídos
data_list = []

# Iteramos por cada archivo y su separador asociado
for file_name, sep in constants.nombre_archivos_sabertyt.items():
    file_path = os.path.join(data_dir_base_sbtyt, file_name)

    # Elegimos el motor según el separador
    engine = "python" if sep == "¬" else "c"

    # Diccionario con argumentos comunes y condicionales

    read_csv_args = {
        "sep": sep,
        "usecols": lambda c: c.lower() in cols_set, # Solo cargar columnas cuyo nombre (en minúscula) esté en la lista deseada
        "engine": engine,
        **(
            {"quotechar": None, "quoting": csv.QUOTE_NONE, "dtype": str}  # Si el separador es '¬', desactivar manejo de comillas y forzar lectura como texto
            if sep == "¬" else
            {"low_memory": False} # Para otros separadores, mejorar manejo de tipos con low_memory=False
        )
    }
    try:
        # Leer el archivo con los argumentos definidos
        df = pd.read_csv(file_path, **read_csv_args)

        # Convertir todos los nombres de columnas a minúsculas
        df.columns = df.columns.str.lower()

        # Verificar si faltan columnas esperadas y avisar
        faltantes = cols_set - set(df.columns)
        if faltantes:
            print(f"Atención: en {file_name} faltan columnas: {faltantes}")

        # Agregar el DataFrame leído a la lista
        data_list.append(df)

    except Exception as err:
        # Reportar errores durante la lectura
        print(f"Error leyendo {file_name} con separador '{sep}': {err}")


# Combinar todos los DataFrames en uno solo
base_sbtyt = pd.concat(data_list, ignore_index=True)
#agregar la columna punt_global porque el icfes no la incluyó en la actualizacion del 31/07/2025
base_sbtyt = calcular_punt_global(base_sbtyt)


In [7]:
resumen_nulos(base_sbtyt)

,n_nulos,porc_nulos
mod_ingles_punt,679.0,0.088152
mod_competen_ciudada_punt,0.0,0.000000
mod_comuni_escrita_punt,900.0,0.116843
mod_razona_cuantitat_punt,0.0,0.000000
mod_lectura_critica_punt,0.0,0.000000


### Ajustes de formato 

Tipado de variables

In [8]:
#Parse las columnas a los tipos de variables adecuados
for col, dtype in constants.dtype_mapping_sabertyt.items():
    if col in base_sbtyt.columns:
        try:
            base_sbtyt[col] = base_sbtyt[col].astype(dtype)
        except Exception as e:
            print(f"No se pudo convertir la columna {col} a {dtype}: {e}")


cols_numericas = [
    "periodo",
    "inst_cod_institucion",
    "estu_snies_prgmacademico",
    "estu_prgm_codmunicipio"
]        
#parse a numerico las variables numericas
for col in cols_numericas:
    base_sbtyt[col] = pd.to_numeric(base_sbtyt[col], errors='coerce').astype('Int64')

Ajuste de variables

In [9]:
#Ajustar los periodos para agregarlos anualmente
base_sbtyt['año_presentacion'] = base_sbtyt['periodo'].apply(lambda x: x//10)

### Filtrar por Bogota-Region

In [10]:
#Codigos y nombre Municipios Bogotá Region
municipios_bogota_region = pd.read_csv("../../data/Municipios_cleaned/municipios.csv")

#Quedarse con las observaciones del saber tyt donde el programa educativo esta ubicado en Bogota-Region 
base_sbtyt = base_sbtyt[base_sbtyt["estu_prgm_codmunicipio"].isin(municipios_bogota_region["codigo_dane_municipio"])]


In [11]:
base_sbtyt = base_sbtyt.sort_values(by="periodo")
base_sbtyt = base_sbtyt.reset_index(drop=True)

print(f"Dimensiones: {base_sbtyt.shape}")
base_sbtyt.head()

Dimensiones: (248425, 17)


,periodo,estu_consecutivo,estu_nivel_prgm_academico,estu_nucleo_pregrado,estu_prgm_academico,estu_prgm_codmunicipio,estu_prgm_municipio,estu_snies_prgmacademico,inst_cod_institucion,inst_nombre_institucion,mod_competen_ciudada_punt,mod_comuni_escrita_punt,mod_ingles_punt,mod_lectura_critica_punt,mod_razona_cuantitat_punt,punt_global,año_presentacion
0,20201,EK202010043695,TECNOLOGÍA,ADMINISTRACIÓN,TECNOLOGIA EN REGENCIA DE FARMACIA,11001,BOGOTÁ D.C.,2206,3808,CORPORACION TECNOLOGICA DE BOGOTA - C.T.B.-BOG...,112.0,76.0,105.0,70.0,74.0,87.4,2020
1,20201,EK202010087792,TECNOLOGÍA,"INGENIERÍA DE SISTEMAS, TELEMÁTICA Y AFINES",TECNOLOGIA EN ANALISIS Y DESARROLLO DE SISTEMA...,11001,BOGOTÁ D.C.,107357,9110,SERVICIO NACIONAL DE APRENDIZAJE-SENA,140.0,92.0,148.0,145.0,127.0,130.4,2020
2,20201,EK202010023524,TECNOLOGÍA,INGENIERÍA MECÁNICA Y AFINES,TECNOLOGÍA EN DISEÑO E INTEGRACIÓN DE AUTOMATI...,11001,BOGOTÁ D.C.,91169,9110,SERVICIO NACIONAL DE APRENDIZAJE-SENA,129.0,112.0,116.0,131.0,106.0,118.8,2020
3,20201,EK202010033363,TÉCNICO PROFESIONAL,ECONOMÍA,TÉCNICO PROFESIONAL EN COMERCIO EXTERIOR,11001,BOGOTÁ D.C.,103188,2728,FUNDACION UNIVERSITARIA DEL AREA ANDINA-BOGOTÁ...,52.0,200.0,167.0,61.0,106.0,117.2,2020
4,20201,EK202010082429,TECNOLOGÍA,INGENIERÍA INDUSTRIAL Y AFINES,TECNOLOGÍA EN LOGÍSTICA,11001,BOGOTÁ D.C.,101388,2725,POLITECNICO GRANCOLOMBIANO-BOGOTÁ D.C.,144.0,117.0,139.0,115.0,112.0,125.4,2020


### Estadisticas descriptivas

In [12]:
num_programas_unicos  = base_sbtyt['estu_snies_prgmacademico'].nunique()
num_ies_unicas  = base_sbtyt['inst_cod_institucion'].nunique()
num_estudiantes_unicos = base_sbtyt['estu_consecutivo'].nunique()
periodos_unicos = base_sbtyt['periodo'].unique()
años_unicos = base_sbtyt['año_presentacion'].unique()

In [13]:
print(
    f"Numero programas unicos: {num_programas_unicos} \n" 
    f"Numero IES unicas: {num_ies_unicas}\n"
    f"Numero estudiantes unicos: {num_estudiantes_unicos} de {base_sbtyt.shape[0]}\n"
    f"Periodos presentacion saber TYT:{periodos_unicos}\n"
    f"Años presentacion saber TYT:{años_unicos}\n"
    )

Numero programas unicos: 1031 
Numero IES unicas: 94
Numero estudiantes unicos: 248425 de 248425
Periodos presentacion saber TYT:<IntegerArray>
[20201, 20202, 20203, 20211, 20212, 20213, 20221, 20223, 20224, 20226, 20231,
 20232, 20233, 20234, 20241, 20242, 20243, 20244]
Length: 18, dtype: Int64
Años presentacion saber TYT:[2020 2021 2022 2023 2024]



### Guardar dataframe

In [23]:
from datetime import datetime

# Generate today's date string in DDMMYYYY format
fecha_hoy = datetime.today().strftime('%d%m%Y')
# Build the full path with the dynamic date
nombre_archivo = f"../../data/SABERTYT_cleaned/base_sbtyt_bogota_region_corte_{fecha_hoy}.csv"

# Save the file
base_sbtyt.to_csv(nombre_archivo, index=False)